In [3]:
import re
import random
import time
import json
import tabula
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from selenium import webdriver
from sklearn.neighbors import KernelDensity
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

# Scraping

In [4]:
def fixNames(names):
    # Lista para guardar los nombres reordenados
    reordered_list = []
    
    for name in names:        
        reordered_name = ""
        split_string = name.split(' ')
        if len(split_string) == 3:
            if split_string[0][1].isupper():
                if split_string[1][1].isupper():
                    reordered_name = split_string[0] + " " + split_string[1] + " " + split_string[2]
                elif split_string[2][1].isupper():
                    reordered_name = split_string[0] + " " + split_string[2] + " " + split_string[1]
                else:
                    reordered_name = split_string[0] + " " + split_string[1] + " " + split_string[2]
            elif split_string[1][1].isupper():
                if split_string[2][1].isupper():
                    reordered_name = split_string[1] + " " + split_string[2] + " " + split_string[0]
                else:
                    reordered_name = split_string[1] + " " + split_string[0] + " " + split_string[2]
            else:
                reordered_name = split_string[2] + " " + split_string[0] + " " + split_string[1]
        else:
            if split_string[0][1].isupper():
                reordered_name = split_string[0] + " " + split_string[1]
            else:
                reordered_name = split_string[1] + " " + split_string[0]
        
        # Añade el nombre reordenado a la lista
        reordered_list.append(reordered_name)
        
    return reordered_list
    
    

In [5]:
def scraping_names_from_pdf():
    # Lee el archivo PDF como un DataFrame de pandas
    dfM = pd.read_excel("Men9Jul.xlsx")
    dfW = pd.read_excel("Women9Jul.xlsx")
    
    # Guarda en singleMen todas las filas que tengan una 'x' en la columna Singles
    # y de cada fila solo guarda lo que esta en la columna Athlete (el nombre)
    singleMen = dfM[dfM['SINGLES'] == 'x']['ATHLETE'].tolist()
    with open('singleMen.json', 'w') as f:
        json.dump(fixNames(singleMen), f)
    
    # Guarda en teamsMen todas las filas que tengan una 'x' en la columna Teams
    # y de cada fila solo guarda lo que esta en la columna Athlete (el nombre)
    teamsMen = dfM[dfM['TEAMS'] == 'x']['ATHLETE'].tolist()
    with open('teamsMen.json', 'w') as f:
        json.dump(fixNames(teamsMen), f)
    
    # Guarda en singleWomen todas las filas que tengan una 'x' en la columna Singles
    # y de cada fila solo guarda lo que esta en la columna Athlete (el nombre)
    singleWomen = dfW[dfW['SINGLES'] == 'x']['ATHLETE'].tolist()
    with open('singleWomen.json', 'w') as f:
        json.dump(fixNames(singleWomen), f)
    
    # Guarda en teamsWomen todas las filas que tengan una 'x' en la columna Teams
    # y de cada fila solo guarda lo que esta en la columna Athlete (el nombre)
    teamsWomen = dfW[dfW['TEAMS'] == 'x']['ATHLETE'].tolist()
    with open('teamsWomen.json', 'w') as f:
        json.dump(fixNames(teamsWomen), f)
    
    # Guarda en mixed todas las filas que tengan una 'x' en la columna Mixed doubles
    # y de cada fila solo guarda lo que esta en la columna Athlete (el nombre)
    mixedW = dfW[dfW['MIXED DOUBLES'] == 'x']['ATHLETE'].tolist()
    mixedWCON = dfW[dfW['MIXED DOUBLES'] == 'x']['NOC'].tolist()
    fixMixedW = fixNames(mixedW)
    mixedWfinal = {}
    for name, noc in zip(fixMixedW, mixedWCON):
        mixedWfinal[name] = noc
    with open('mixedW.json', 'w') as f:
        json.dump(mixedWfinal, f)
        
        
    mixedM = dfM[dfM['MIXED DOUBLES'] == 'x']['ATHLETE'].tolist()
    mixedMCON = dfM[dfM['MIXED DOUBLES'] == 'x']['NOC'].tolist()
    fixMixedM = fixNames(mixedM)
    mixedMfinal = {}
    for name, noc in zip(fixMixedM, mixedMCON):
        mixedMfinal[name] = noc
    with open('mixedM.json', 'w') as f:
        json.dump(mixedMfinal, f)
    
    
    selectedW = dfW[dfW['ATHLETE'].str.len() > 1]['ATHLETE'].tolist()
    selectedM = dfM[dfM['ATHLETE'].str.len() > 1]['ATHLETE'].tolist()
    
    selectedAll = selectedW + selectedM
    
    fixSelectedAll = fixNames(selectedAll)
    
    with open('clasificated.json', 'w') as f:
        json.dump(fixSelectedAll, f)
        
    if selectedAll is None or fixSelectedAll is None:
        print("Error: 'selectedAll' o 'fixselall' no han sido inicializados.")
    else:
        # Lista para guardar los nombres con ranking
        ranking_list = []

        # Usamos zip() para combinar selectedAll y fixselall en pares de elementos
        for name, fixName in zip(selectedAll, fixSelectedAll):
            try:
                # Intentamos obtener el ranking de dfM
                rank = dfM[dfM['ATHLETE'].str.len() > 1]['RANKING'].tolist()[selectedM.index(name)]
            except ValueError:
                # Si no está en dfM, intentamos obtener el ranking de dfW
                rank = dfW[dfW['ATHLETE'].str.len() > 1]['RANKING'].tolist()[selectedW.index(name)]
            
            # Añade el nombre con el ranking a la lista
            ranking_list.append((fixName, rank))

                
    array_serializable = {t[0]: t[1] for t in ranking_list}
    
    with open('clasificated_ranking.json', 'w') as f:
        json.dump(array_serializable, f)

In [7]:
scraping_names_from_pdf()

In [10]:
def results(name1, driver):
    
    table_data = []
    
    wait = WebDriverWait(driver, 15)    # Espera a que el botón esté visible o pasen 5 segundos
    
    playerA = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="search"][name="fabrik_list_filter_all_29_com_fabrik_29"]')))
    playerA.clear()
    playerA.send_keys(name1)
    playerA.send_keys(Keys.RETURN)
    

    time.sleep(5)
    
    table = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "table.table.table-striped.table-condensed.table-sm")))
    rows = table.find_elements(By.XPATH, ".//tbody/tr")
    
    # Extrae los encabezados de la tabla
    headers = table.find_elements(By.XPATH, ".//thead/tr/th")
    final_headers = []
    for header1 in headers:
        header2 = header1.find_element(By.XPATH, ".//span")
        final_headers.append(header2.text)
    table_data.append(final_headers)
        
    for i in range(1, len(rows)):
        # Crea una lista para almacenar las celdas de la fila
        row_data = []
        
        cells = rows[i].find_elements(By.XPATH, ".//td")
        
        for cell in cells:
            # Añade el texto de la celda a la lista de la fila
            row_data.append(cell.text)
        
        # Añade la lista de la fila al array de la tabla
        table_data.append(row_data)
        
    return table_data


In [26]:
def scraping():    # Scrapear los datos
    driver = webdriver.Chrome()    # Inicio el navegador 
    wait = WebDriverWait(driver, 10)    # Espera a que el botón esté visible o pasen 10 segundos

    driver.get('https://results.ittf.link/index.php')    # Me dirijo a la pagina correspondiente
    
    driver.maximize_window()    # Maximizar la ventana del navegador
    
    accept_cookies = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'jb-accept')))
    accept_cookies.click()
    
    username = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="text"][name="username"]')))
    username.send_keys('Selenium1')
    
    password = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="password"][name="password"]')))
    password.send_keys('Selenium1**')

    bottom_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.gotobottom")))
    bottom_button.click()
    
    time.sleep(2)

    continue_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[name='Submit']")))
    continue_button.click()    # Haz clic en el botón "Log in"
    
    arrow = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Matches']")))
    arrow.click()
    
    time.sleep(30)
        
    # link = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@href="/index.php/matches/players-matches"]')))
    # link.click()
    
    select_element = wait.until(EC.element_to_be_clickable((By.ID, "limit29")))
    select_element.click()
    
    select = Select(select_element)
    select.select_by_index(len(select.options)-1)
    
    names = []
    with open('clasificated.json', 'r') as f:
        names = json.load(f)
    
    table_data = []
    diccionario = {}
    
    for i in range(len(names)):
        table_data = results(names[i], driver)
        diccionario[names[i]] = table_data
    
    # Guardar el diccionario convertido en el archivo JSON
    with open("data.json", "w") as archivo_json:
        json.dump(diccionario, archivo_json, indent=4)
    
    driver.quit()    # Cierro el navegador
    return

In [60]:
scraping()

# Proceso Estadistico

In [2]:
def clear_data(data):
    for key in data:
        for registro in data[key]:
            for i in [2, 3, 4, 5]:
                if registro[i] != "":
                    registro[i] = registro[i][:-6]
    return data


In [3]:
def pocos_enfrentamientos(team1, team2, data, event):
    
    conteos = {}
    for team3 in data:
        if team3 in [team1, team2]: continue
        enfrentamientos1, wins1 = stats(team1, team3, event, data, False)
        if enfrentamientos1 == 0: continue
        enfrentamientos2, wins2 = stats(team3, team2, event, data, False)
        if enfrentamientos2 == 0: continue
        
        conteos[team3] = (enfrentamientos1, enfrentamientos2, wins1, wins2)
        
    # Encontrar el atleta con la menor diferencia de enfrentamientos
    atleta_c = None
    mayor_num = -float('inf')
    wins_team1 = 0
    total_enfrentamientos = 0
    for atleta, (enfrentamientos1, enfrentamientos2, wins1, wins2) in conteos.items():
        num = (enfrentamientos1 + enfrentamientos2) / (abs(enfrentamientos1 - enfrentamientos2) + 1)
        if num > mayor_num:
            mayor_num = num
            atleta_c = atleta
            wins_team1 = wins1 + wins2
            total_enfrentamientos = enfrentamientos1 + enfrentamientos2
    z = 1+1
    return total_enfrentamientos, wins_team1 if atleta_c else 0, 0

In [4]:
def define_winner(enfrentamientos, victorias_team1, team1, team2):
    winner1_prob = victorias_team1 / enfrentamientos
    
    numero_aleatorio = random.uniform(0, 1)
    
    if numero_aleatorio < winner1_prob:
        return team1
    else: 
        return team2

In [17]:
def stats(team1, team2, event=None, data=[], rebuscar=True):
    
    enfrentamientos = 0
    victorias_team1 = 0
    
    if event == "S":
        # Verificar si los jugadores existen en el data
        if team1 in data and team2 in data:
            # Iterar sobre los registros de team1
            for registro in data[team1]:
                # Asegurarse de que el registro tiene la longitud adecuada y contiene nombres de jugadores
                if len(registro) > 11 and (team1 in registro or team2 in registro):
                    # Contar enfrentamiento si ambos jugadores están presentes
                    if team1 in registro and team2 in registro:
                        enfrentamientos += 1
                        # Contar victoria para team1
                        if registro[11] == team1 or registro[12] == team1:
                            victorias_team1 += 1
    elif event == "M":
        # Verificar si los jugadores existen en el data
        if team1[0] in data and team2[0] in data and team1[1] in data and team2[1] in data:
            # Iterar sobre los registros de team1
            for registro in data[team1[0]]:
                # Asegurarse de que el registro tiene la longitud adecuada y contiene nombres de jugadores
                if len(registro) > 11 and (team1[0] in registro and team1[1] in registro and team2[0] in registro and team2[1] in registro):
                    enfrentamientos += 1
                    # Contar victoria para team1
                    if (registro[11] == team1[0] and registro[12] == team1[1]) or (registro[11] == team1[1] and registro[12] == team1[0]):
                        victorias_team1 += 1        
                        
    if rebuscar and enfrentamientos < 1:
        x, y = pocos_enfrentamientos(team1, team2, data, event)
        enfrentamientos = x
        victorias_team1 = y
    z = 1+1
    return enfrentamientos, victorias_team1
    


In [6]:
def enfrentamiento(team1, team2, event, data):
    if event == "T":
        # Por equipos: de 5 a ganar 3, cada equipo es de 3 jugadores (ABC vs XYZ)
        #     Doble B+C vs Y+Z
        #     Individual A vs X
        #     Individual C vs Z
        #     Individual A vs Y
        #     Individual B vs X
        
        vt1 = 0
        vt2 = 0
        
        enfrentamientos, victorias_team1 = stats((team1[1], team1[2]), (team2[1], team2[2]), event, data)
        winner1 = define_winner(enfrentamientos, victorias_team1, team1, team2)
        if winner1 == team1: vt1 += 1 
        else: vt2 +=1
        
        enfrentamientos, victorias_team1 = stats(team1[0], team2[0], event, data)
        winner2 = define_winner(enfrentamientos, victorias_team1, team1, team2)
        if winner2 == team1: vt1 += 1 
        else: vt2 +=1
        
        enfrentamientos, victorias_team1 = stats(team1[2], team2[2], event, data)
        winner3 = define_winner(enfrentamientos, victorias_team1, team1, team2)
        if winner3 == team1: vt1 += 1 
        else: vt2 +=1
        
        if vt1 > 2:
            return winner1
        elif vt2 > 2:
            return winner1
        else:
            enfrentamientos, victorias_team1 = stats(team1[0], team2[1], event, data)
            winner4 = define_winner(enfrentamientos, victorias_team1, team1, team2)
            if winner4 == team1: vt1 += 1 
            else: vt2 +=1
            
            if vt1 > 2:
                return winner4
            elif vt2 > 2:
                return winner4
            else:
                enfrentamientos, victorias_team1 = stats(team1[1], team2[0], event, data)
                return define_winner(enfrentamientos, victorias_team1, team1, team2)
    
    else:
        enfrentamientos, victorias_team1 = stats(team1, team2, event, data)
        return define_winner(enfrentamientos, victorias_team1, team1, team2)

In [7]:
def FirstRoundsInd(homeclubs, aways, data):
    winners = []
    for i in range(16):
        team1 = random.choice(homeclubs)
        homeclubs.remove(team1)
        if aways:
            team2 = random.choice(aways)
            aways.remove(team2)
            winners.append(enfrentamiento(team1, team2, "S", data))
        else:
            winners.append(team1)
    return winners

In [8]:
def BracketInd(homes, aways, data):
    number1 = homes[0]
    number2 = homes[1]
    number3to4 = homes[2:4]
    number5to8 = homes[4:9]
    number9to16 = homes[8:]
    
    # Round of 32
    team1 = number1
    team2 = random.choice(aways)
    aways.remove(team2)
    winner1 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner2 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner3 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number5to8)
    number5to8.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner4 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number5to8)
    number5to8.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner5 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner6 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner7 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number3to4)
    number3to4.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner8 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number3to4)
    number3to4.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner9 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner10 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner11 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number5to8)
    number5to8.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner12 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number5to8)
    number5to8.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner13 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner14 = enfrentamiento(team1, team2, "S", data)
    
    team1 = random.choice(number9to16)
    number9to16.remove(team1)
    team2 = random.choice(aways)
    aways.remove(team2)
    winner15 = enfrentamiento(team1, team2, "S", data)
    
    team1 = number2
    team2 = random.choice(aways)
    aways.remove(team2)
    winner16 = enfrentamiento(team1, team2, "S", data)
    
    winners = ["empty", winner1, winner2, winner3, winner4, winner5, winner6, winner7, winner8, winner9, winner10, winner11, winner12, winner13, winner14, winner15, winner16]
    
    return Final16(winners, "S")

In [9]:
def choose(array):
    element = random.choice(array)
    array.remove(element)
    return element

In [10]:
def Final16(winners, event, data):
    if event == "S":
        number1 = winners[0]
        number2 = winners[1]
        number3to4 = winners[2:4]
        number5to8 = winners[4:9]
        number9to16 = winners[8:]
        
        winners = ["empty", number1, choose(number9to16), choose(number9to16), choose(number5to8), choose(number5to8), 
                    choose(number9to16), choose(number9to16), choose(number3to4), choose(number3to4), choose(number9to16), 
                    choose(number9to16), choose(number5to8), choose(number5to8), choose(number9to16), choose(number9to16), number2]
        
    # Round of 16
    print("start")
    print("match")
    winner1R16 = enfrentamiento(winners[1], winners[2], event, data)
    print("match")
    winner2R16 = enfrentamiento(winners[3], winners[4], event, data)
    print("match")
    winner3R16 = enfrentamiento(winners[5], winners[6], event, data)
    print("match")
    winner4R16 = enfrentamiento(winners[7], winners[8], event, data)
    print("match")
    winner5R16 = enfrentamiento(winners[9], winners[10], event, data)
    print("match")
    winner6R16 = enfrentamiento(winners[11], winners[12], event, data)
    print("match")
    winner7R16 = enfrentamiento(winners[13], winners[14], event, data)
    print("match")
    winner8R16 = enfrentamiento(winners[15], winners[16], event, data)
    
    # Quarterfinals
    print("match")
    winner1QF = enfrentamiento(winner1R16, winner2R16, event, data)
    print("match")
    winner2QF = enfrentamiento(winner3R16, winner4R16, event, data)
    print("match")
    winner3QF = enfrentamiento(winner5R16, winner6R16, event, data)
    print("match")
    winner4QF = enfrentamiento(winner7R16, winner8R16, event, data)
    
    # Semifinals
    print("match")
    winner1SF = enfrentamiento(winner1QF, winner2QF, event, data)
    print("match")
    winner2SF = enfrentamiento(winner3QF, winner4QF, event, data)
    
    # Final
    print("match")
    FinalWinner = enfrentamiento(winner1SF, winner2SF, event, data)
    
    return FinalWinner

In [23]:
def singles(genre):
    with open('data.json', 'r') as f:
            dataf = json.load(f)
    dataJson = clear_data(dataf)
    
    with open(f'single{genre}.json', 'r') as f:
        data = json.load(f)
    with open(f'clasificated_ranking.json', 'r') as f:
        info = json.load(f)
    datos = {}
    for name in data:
        datos[name] = info[name]
        
    diccionario_ordenado = dict(sorted(datos.items(), key=lambda item: item[1]))
    
    datos = list(diccionario_ordenado.keys())
    
    aways16 = FirstRoundsInd(datos[16:32], FirstRoundsInd(datos[32:48], FirstRoundsInd(datos[48:64], datos[64:], dataJson), 
                                                            dataJson), 
                            dataJson)

    Final16(datos[:16], aways16)

print(singles("Men"))
print(singles("Women"))

ValueError: too many values to unpack (expected 2)

In [ ]:
def teams(genre):
    with open(f'teams{genre}.json', 'r') as f:
        data = json.load(f)
    with open(f'clasificated_ranking.json', 'r') as f:
        info = json.load(f)
    
    info_tupla = list(zip(data[::3], data[1::3], data[2::3]))
    datos = {}
    # Corregimos el bucle para sumar los valores de ranking de cada nombre en la tupla
    for name in info_tupla:
        # Sumamos los valores de ranking para cada nombre en la tupla
        datos[name] = sum(info[nombre] for nombre in name if nombre in info)
        
    diccionario_ordenado = dict(sorted(datos.items(), key=lambda item: item[1]))
    
    datos = list(diccionario_ordenado.keys())

    Final16(datos[:16], "T")

print(teams("Men"))
print(teams("Women"))

start
match


KeyboardInterrupt: 

In [ ]:
x = ["JEE Minhyung", "TAPPER Melissa", "BROMLEY Michelle", "SUN Yingsha", "CHEN Meng", "WANG Manyu", "MESHREF Dina", "GODA Hana", "ALHODABY Mariam", "MITTELHAM Nina", "SHAN Xiaona", "KAUFMANN Annett", "ZHANG Lily", "WANG Amy", "SUNG Rachel", "TAKAHASHI Bruna", "TAKAHASHI Giulia", "ALEXANDRE Bruna", "SHIN Yubin", "JEON Jihee", "LEE Eunhye", "PAVADE Prithika", "YUAN Jia Nan", "LUTZ Charlotte", "CHENG I-Ching", "CHIEN Tung-Chuan", "CHEN Szu-Yu", "DOO Hoi Kem", "ZHU Chengzhu", "LEE Ho Ching", "SZOCS Bernadette", "SAMARA Elizabeta", "DIACONU Adina", "HAYATA Hina", "HIRANO Miu", "HARIMOTO Miwa", "PARANANG Orawan", "SAWETTABUT Suthasini", "SAWETTABUT Jinnipa", "BAJOR Natalia", "WEGRZYN Katarzyna", "WIELGOS Zuzanna", "BATRA Manika", "AKULA Sreeja", "KAMATH Archana Girish", "KALLBERG Christina", "BERGSTROM Linda", "BERGAND Filippa", "FONSECA CARRAZANA Daniela", "KIM Kun Yong", "XIAO Maria", "MADARASZ Dora", "TOMMY Priscilla", "ZENG Jian", "ALI Fathimath", "ZHANG Mo", "VEGA Paulina", "ZENG Zhiying", "COSSIO ACEVES Arantxa", "EDEM Offiong", "HANFFOU Sarah", "BELLO Fatimo", "SHAHSAVARI Neda", "SAHAKIAN Mariana", "HURSEY Anna", "EERLAND Britt", "SHAO Jieni", "YU Fu", "PESOTSKA Margaryta", "PYON Song Gyong", "LOGHRAIBI Lynda", "ZHOU Jingyi", "YANG Xiaoxin", "DIAZ Adriana", "EDGHILL Chelsea", "POLCANOVA Sofia", "NI Xia Lian", "PICCOLIN Giorgia", "POTA Georgina", "MALOBABIC Ivana", "MATELOVA Hana", "LUPULESKU Izabela", "VIVARELLI Debora", "DE NUTTE Sarah", "ALTINKAYA Sibel", "BRATEYKO Solomiya", "LUM Nicholas", "LUU Finn", "BAE Hwan", "WANG Chuqin", "FAN Zhendong", "MA Long", "KALLBERG Anton", "MOREGARD Truls", "KARLSSON Kristian", "LY Edward", "WANG Eugene", "HAZIN Jeremy", "CALDERANO Hugo", "ISHIY Vitor", "TEODORO Guilherme", "ASSAR Omar", "EL-BEIALI Mohamed", "ABDEL-AZIZ Youssef", "HARIMOTO Tomokazu", "TOGAMI Shunsuke", "SHINOZUKA Hiroto", "GROTH Jonathan", "LIND Anders", "ANDERSEN Martin Buch", "JANG Woojin", "CHO Daeseong", "LIM Jonghoon", "LEBRUN Felix", "LEBRUN Alexis", "GAUZY Simon", "FREITAS Marcos", "APOLONIA Tiago", "GERALDO Joao", "LIN Yun-Ju", "KAO Cheng-Jui", "CHUANG Chih-Yuan", "QIU Dang", "OVTCHAROV Dimitrij", "BOLL Timo", "JORGIC Darko", "KOZUL Deni", "HRIBAR Peter", "PUCAR Tomislav", "GACINA Andrej", "ZELJKO Filip", "ACHANTA Sharath Kamal", "DESAI Harmeet", "THAKKAR Manav Vikash", "CAMPOS Jorge", "RI Jong Sik", "WONG Chun Ting", "ROBLES Alvaro", "IONESCU Ovidiu", "ECSEKI Nandor", "QUEK Izaac", "SHRESTHA Santoo", "LORENZO Santiago", "MADRID Marcos", "PEREIRA Andy", "JHA Kanak", "OMOTAYO Olajide", "RAKOTOARIMANANA Fabio", "BOULOUSSA Mehdi", "ALAMIAN Nima", "ABO YAMAN Zaid", "REDZIMSKI Milosz", "GIONIS Panagiotis", "URSU Vladislav", "ZHMUDENKO Yaroslav", "YANG Wang", "BURGOS Nicolas", "GERASSIMENKO Kirill", "ARUNA Quadri", "WU Vicky", "IDOWU Saheed", "PITCHFORD Liam", "ALAMIYAN Noshad", "HABESOHN Daniel", "IONESCU Eduard", "AFANADOR Brian", "ALLEGRO Martin", "MINO Alberto", "DIAW Ibrahima", "GONZALEZ Daniel", "MLADENOVIC Luka", "NUYTINCK Cedric"]

with open('data.json', 'r') as f:
    dataf = json.load(f)


for y in x:
    if len(dataf[y]) <= 1:
        print(y)




In [ ]:
def test(link):
    
    driver = webdriver.Chrome()    # Inicio el navegador 
    wait = WebDriverWait(driver, 10)    # Espera a que el botón esté visible o pasen 10 segundos
    
    driver.get('https://results.ittf.link/index.php')    # Me dirijo a la pagina correspondiente
    
    driver.maximize_window()    # Maximizar la ventana del navegador
    
    accept_cookies = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'jb-accept')))
    accept_cookies.click()
    
    username = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="text"][name="username"]')))
    username.send_keys('Selenium1')
    
    password = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="password"][name="password"]')))
    password.send_keys('Selenium1**')

    bottom_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.gotobottom")))
    bottom_button.click()
    
    time.sleep(2)

    continue_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[name='Submit']")))
    continue_button.click()    # Haz clic en el botón "Log in"
    
    table_data = []

    driver.get(link)    # Me dirijo a la pagina correspondiente
    
    time.sleep(15)

    table = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "table.table.table-striped.table-condensed.table-sm")))
    rows = table.find_elements(By.XPATH, ".//tbody/tr")

    # Extrae los encabezados de la tabla
    headers = table.find_elements(By.XPATH, ".//thead/tr/th")
    final_headers = []
    for header1 in headers:
        header2 = header1.find_element(By.XPATH, ".//span")
        final_headers.append(header2.text)
    table_data.append(final_headers)
        
    for i in range(1, len(rows)):
        # Crea una lista para almacenar las celdas de la fila
        row_data = []
        
        cells = rows[i].find_elements(By.XPATH, ".//td")
        
        for cell in cells:
            # Añade el texto de la celda a la lista de la fila
            row_data.append(cell.text)
        
        # Añade la lista de la fila al array de la tabla
        table_data.append(row_data)
        
    driver.quit()
    return table_data

with open('data.json', 'r') as f:
    data = json.load(f)
    

data['KIM Kun Yong'] = test("https://results.ittf.link/index.php?option=com_fabrik&view=list&listid=31&Itemid=250&resetfilters=1&abc=136607&vw_matches___player_a_id[value][]=136607&vw_matches___player_a_id[join][]=OR&vw_matches___player_x_id[value][]=136607&vw_matches___player_x_id[join][]=OR&vw_matches___player_x_id[grouped_to_previous][]=1&vw_matches___wo[value][]=0&vw_matches___kind[value][]=1")
data['YANG Wang'] = test("https://results.ittf.link/index.php?option=com_fabrik&view=list&listid=31&Itemid=250&resetfilters=1&abc=112735&vw_matches___player_a_id[value][]=112735&vw_matches___player_a_id[join][]=OR&vw_matches___player_x_id[value][]=112735&vw_matches___player_x_id[join][]=OR&vw_matches___player_x_id[grouped_to_previous][]=1&vw_matches___wo[value][]=0&vw_matches___kind[value][]=1")
data['IONESCU Eduardo'] = test("https://results.ittf.link/index.php/player-matches/list/31?resetfilters=0&abc=135023&clearordering=0&clearfilters=0")

data['IONESCU Eduard'] = data.pop('IONESCU Eduardo')

print(len(data['KIM Kun Yong']))
print(len(data['YANG Wang']))
print(len(data['IONESCU Eduard']))


34
101
101


In [ ]:
print(len(data['KIM Kun Yong']))
print(len(data['YANG Wang']))
print(len(data['IONESCU Eduard']))

with open("data.json", "w") as archivo_json:
        json.dump(data, archivo_json, indent=4)

34
101
101
